<a href="https://colab.research.google.com/github/h2llo/h2llo/blob/Thisismain/crfsuite%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pprint import pprint

with open('/content/drive/MyDrive/Colab Notebooks/data/134963_norm.txt', encoding='utf-8') as f:
    docs = [doc.strip() for doc in f]

print('n docs = %d' % len(docs))
pprint(docs[:5])

n docs = 15603
['시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다',
 '사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고',
 '지금껏 영화 평가 해본 적이 없는데 진짜 최고네요 색감 스토리 음악 연기 모두ㅜㅜ최고입니다',
 '방금 시사회 보고 왔어요 배우들 매력이 눈을 뗄 수가 없게 만드네요 한편의 그림 같은 장면들도 많고 음악과 춤이 눈과 귀를 사로 잡았어요 '
 '한번 더 보고 싶네요',
 '초반부터 끝까지 재미있게 잘보다가 결말에서 고국마 왕창먹음 힐링 받는 느낌들다가 막판에 기분 잡쳤습니다 마치 감독이 하고싶은 말은 '
 '너희들이 원하는 결말은 이거지 하지만 현실은 이거다 라고 말하고 싶었나보군요']


In [ ]:
pip install pycrfsuite_spacing

In [ ]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '../',
 '../',
 '../']

In [ ]:
import sys
sys.path.append('../') #sys.path : 파이썬 라이브러리들이 설치되어있는 디렉터리들을 보여준다. 
                       #append 는 추가, '../' 모듈을 추가 해준다는 말임.

from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_chartags
from pycrfsuite_spacing import sent_to_xy

TemplateGenerator와 FeatureTransformer를 이용하면 띄어쓰기가 포함된 문장을 다음과 같이 template에 의하여 정의되는 character feature로 변환해 줍니다. 

sent_to_chars는 띄어쓰기가 있는 문장을 글자와 띄어쓰기 태그로 변환. 띄어쓰기/붙여쓰기의 태그를 지정할 수 있음. default 는 space=1,nonspace=0 으로 int

In [ ]:
chars, tags = sent_to_chartags('이것도 너프해 보시지', space='1', nonspace='0')
print(chars)
print(tags)

이것도너프해보시지
['0', '0', '1', '0', '0', '1', '0', '0', '1']


In [ ]:
sent_to_chartags('')

('', [])

빈 문자가 입력되면 빈 글자와 empty list 가 변환

In [ ]:
TemplateGenerator(begin=-2, end=2, min_range_length=3,max_range_length=3).tolist()

[(-2, 0), (-1, 1), (0, 2)]

TemplateGenerator 는 앞 2글자부터 뒤 2글자 까지 길이가 3인 templates 를 만든다는 의미.
반드시 현재 글자(index 0)가 포함된 부분만 template으로 만듬

In [ ]:
to_feature = CharacterFeatureTransformer(
    TemplateGenerator(begin=-2, 
    end=2,
    min_range_length=3,
    max_range_length=3)
)

x, y = sent_to_xy('이것도 너프해 보시지', to_feature)
pprint(x)
print(y)

[['X[0,2]=이것도'],
 ['X[-1,1]=이것도', 'X[0,2]=것도너'],
 ['X[-2,0]=이것도', 'X[-1,1]=것도너', 'X[0,2]=도너프'],
 ['X[-2,0]=것도너', 'X[-1,1]=도너프', 'X[0,2]=너프해'],
 ['X[-2,0]=도너프', 'X[-1,1]=너프해', 'X[0,2]=프해보'],
 ['X[-2,0]=너프해', 'X[-1,1]=프해보', 'X[0,2]=해보시'],
 ['X[-2,0]=프해보', 'X[-1,1]=해보시', 'X[0,2]=보시지'],
 ['X[-2,0]=해보시', 'X[-1,1]=보시지'],
 ['X[-2,0]=보시지']]
['0', '0', '1', '0', '0', '1', '0', '0', '1']


to_feature : template을 이용하여 문장에서 features를 만드는 함수. CharacterFeatureTransformer의 instance는 callable이다.

In [ ]:
from pycrfsuite_spacing import PyCRFSuiteSpacing

correct = PyCRFSuiteSpacing(
    to_feature = to_feature,
    feature_minfreq=3, # default = 0
    max_iterations=100,
    l1_cost=1.0,
    l2_cost=1.0
)
correct.train(docs, '/content/drive/MyDrive/Colab Notebooks/data/demo_model.crfsuite')

In [ ]:
#PyCRFSuiteSpacing 모델 학습.
#arguments
#feautre_minfreq -> 기본값은 0, 지나치게 작으면 features 개수가 커짐. 메모리가 많이 필요하니 적절한 수준으로 잡아준다.
#max_itrations -> 기본값은 100
#l1_cost,l2_cost
#기본값은 각각 0, Lasso CRF 를 이용하려면 l2_cost=0,l1_cost>0 으로 설정.
#L1,L2 reqularization 을 모두 이용하려면 l1_cost>0 & l2_cost >0 으로 설정.

In [ ]:
#학습된 모델 demo_model.crfsuite 불러오기 load_tagger

In [ ]:
#correct.load_tagger('/content/drive/MyDrive/Colab Notebooks/data/demo_model.crfsuite')

In [ ]:
correct('이건진짜좋은영화라라랜드진짜좋은영화')

'이건 진짜 좋은 영화 라라랜드 진짜 좋은 영화'

In [ ]:
correct('나는오늘짝궁과밥을먹었다')

'나는 오늘짝궁과밥을먹었다'

In [ ]:
correct('하이빅스비')

'하이빅스비'

In [ ]:
correct('이것도너프해보시지')

'이것도 너프해보시지'

In [ ]:
correct(docs[1])

'사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고'

In [ ]:
correct('사랑과꿈그흐름의아름다움을음악과영상으로최대한담아놓았다배우들연기는두말할것없고')

'사랑과 꿈그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고'

In [ ]:
correct('사랑과꿈그흐름의아름다움을음악과영상으로최대한담아놓았다배우들연기는두말할것없고')

'사랑과 꿈그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고'

In [ ]:
from pycrfsuite_spacing import PyCRFSuiteSpacing

model_path = '/content/drive/MyDrive/Colab Notebooks/data/demo_model.crfsuite'
correct = PyCRFSuiteSpacing(to_feature)
correct.train(docs, model_path)

In [ ]:
#https://github.com/lovit/soyspacing/blob/master/tutorials/count-base_spacing_tutorial.ipynb